# Logic in Sole.jl

In [ ]:
using Pkg
Pkg.activate("..")
Pkg.instantiate()
Pkg.update()

## SoleLogics.jl

`SoleLogics.jl` is not only the package in the `Sole.jl` framework specifically
developed for logic: it is the core library of the framework itself.

In a nutshell, it provides a fresh codebase for computational logic, featuring
easy manipulation of:
- Propositional and (multi)modal logics (atoms, logical constants, alphabets,
grammars, crisp/fuzzy algebras);
- Logical formulas (parsing, random generation, minimization);
- Logical interpretations (propositional valuations, Kripke structures);
- Algorithms for finite
[model checking](https://en.wikipedia.org/wiki/Model_checking), that is,
checking that a formula is satisfied by an interpretation.

In this notebook, we will see examples of all these functionalities, providing
a comprehensive overview of the whole package.

In [ ]:
using SoleLogics

### Propositional Logic

#### Formulas and Intepretations

In [ ]:
p = Atom("it's alive!")

In [ ]:
q = Atom("it's mortal!")

In [ ]:
φ = p ∧ q

In [ ]:
φ isa Formula && p isa Formula

φ is the root node of a syntax tree.

In [ ]:
token(φ)    # Print the syntax token at the root node

In [ ]:
children(φ) # Print the children of the root node

Let's create a method for negating any formula.

In [ ]:
function negateformula(f::Formula)
    return ¬f
end

In [ ]:
negateformula(φ)

In [ ]:
negateformula(p)

We can use `syntaxstring` to obtain the string representation of a `Formula`.

In [ ]:
syntaxstring(φ)

We can also parse `Formula`s from standard string representations.

In [ ]:
φ = parseformula("it's alive! ∧ it's mortal!")

Let's see how we can build our own string representation.

*Spoiler*: it's a good example of Julia's multiple dispatch in action!

In [ ]:
function my_own_string_representation(f::Atom)
    return syntaxstring(f)
end

function my_own_string_representation(::NamedConnective{:¬}, children)
    return "It is not the case that $(my_own_string_representation(first(children)))"
end

function my_own_string_representation(::NamedConnective{:∧}, children)
    subformula1, subformula2 = children
    return "both $(my_own_string_representation(subformula1)) and $(my_own_string_representation(subformula2))"
end

function my_own_string_representation(f::SyntaxBranch)
    return my_own_string_representation(token(f), children(f))
end

In [ ]:
my_own_string_representation(¬φ)

Use use `TruthDict` structures (truth dictionaries, or dictionaries of truths)
to associate to each atom il our alphabet a truth value.

You can see these as properties of an object, or features/attributes of an
instance!

In [ ]:
soul = TruthDict([p => true, q => false])

In [ ]:
body = TruthDict([p => true, q => true])

But what is ⊤?

In [ ]:
⊤ isa Truth      &&     ⊤ isa Formula

If you find these Unicode characters uncomfortable to work with, they have
aliases:

In [ ]:
(TOP, BOT, CONJUNCTION, DISJUNCTION, IMPLICATION)

Now we can check if a model or interpretation, represented by a `TruthDict`,
satisfies a `Formula`!

In [ ]:
check(p, soul)  # soul is alive

In [ ]:
check(φ, soul)  # but not both alive and mortal

In [ ]:
check(φ, body)  # body is both alive and mortal!

These objects can actually be used as dictionaries from `Formula` to `Truth`
values.

For example, we both assign ⊤ (top) to the atom "alive".

In [ ]:
soul[p], body[p]

Not only, indexing can be used to check generic `Formula`s.

In [ ]:
soul[φ]

This is syntactic sugar for the *interpretation* algorithm, which is actually
more general than check!

In [ ]:
interpret(φ, soul)

In fact, it also works under incomplete information.

In [ ]:
body[φ ∧ Atom("?Unknown property?")]

Notice how in this example, with an *unknown atom*, it uses the *known*
information to simplify the formula

In [ ]:
body[φ ∨ Atom("?Unknown property?")]

So ultimately, `check` is just a shortcut for making sure that `interpret`
simplifies the formula to ⊤.

In [ ]:
check(φ, soul) == istop(interpret(φ, soul))

Let's generate random formulas!

In [ ]:
SoleLogics.BASE_PROPOSITIONAL_CONNECTIVES

In [ ]:
Σ = @atoms a b

In [ ]:
using Random

h = 2;   # the height of the formula

φ = randformula(
    Random.MersenneTwister(300),    # the random number generator we want to use
    h,
    Σ,
    SoleLogics.BASE_PROPOSITIONAL_CONNECTIVES
)

In [ ]:
syntaxstring(φ; parenthesize_commutatives = true)

In [ ]:
normalize(φ)

**Exercise**: Check many, randomly-generated formulas on the alphabet ${p, q}$
on both `soul` and `body`. Do `soul` and `body` have the same probability of
satisfying a generic formula? Can you estimate this probability?

#### Scalar Interpretations

Now, let's consider a propositional interpretation on scalar attributes
$A_1, A_2, \ldots$, and check formulas on an alphabet
$\mathcal{A} \subseteq \{A_i < v, v \in \mathbb{R}\}$ on it.

We start by defining the atoms of type $A_i < v.$

In [ ]:
import SoleLogics: syntaxstring

struct ConditionOnAttribute
    i_attribute::Integer
    threshold::Real
end

function syntaxstring(c::ConditionOnAttribute; kwargs...)
    "A$(c.i_attribute) < $(c.threshold)"
end

syntaxstring(ConditionOnAttribute(2, 10))

In [ ]:
using SoleLogics: AbstractAssignment # Abstract type for propositional Interpretations

struct TabularInterpretation{T<:Real} <: AbstractAssignment
    vals::Vector{T}
end

import SoleLogics: interpret, value

function interpret(a::Atom{ConditionOnAttribute}, I::TabularInterpretation)
    cond = value(a)
    return (I.vals[cond.i_attribute] < cond.threshold ? ⊤ : ⊥)
end

In [ ]:
rng = Random.MersenneTwister(1)
n_variables = 4

vals = rand(rng, n_variables)
I = TabularInterpretation(vals)

In [ ]:
A = Atom.(
    [ConditionOnAttribute(v, t) for v in 1:n_variables for t in 0:0.1:1.0]
)
syntaxstring.(A)

In [ ]:
[interpret(cond, I) for cond in A]

In [ ]:
rng = Random.MersenneTwister(32)

[
    begin
        f = randformula(rng, 3, A, SoleLogics.BASE_PROPOSITIONAL_CONNECTIVES)
        syntaxstring(f) => interpret(f, I)
    end for _ in 1:10
]


**Exercise**: Check many, randomly-generated formulas on many,
randomly-generated tabular interpretations, and store the formulas that satisfy
the highest number of instances!

### Modal Logic K

Let's instantiate a frame with 5 worlds and 5 edges.

In [ ]:
using Graphs

worlds = SoleLogics.World.(1:5)
edges = Edge.([(1,2), (1,3), (2,4), (3,4), (3,5)])
fr = SoleLogics.ExplicitCrispUniModalFrame(worlds, Graphs.SimpleDiGraph(edges))

We pick the first world...

In [ ]:
w1 = worlds[1]

...and we enumerate the world that are accessible from the first world.

In [ ]:
accessibles(fr, w1)

Let's assign each world a propositional interpretation.

In [ ]:
valuation = Dict([
    worlds[1] => TruthDict([p => true, q => false]),
    worlds[2] => TruthDict([p => true, q => true]),
    worlds[3] => TruthDict([p => true, q => false]),
    worlds[4] => TruthDict([p => false, q => false]),
    worlds[5] => TruthDict([p => false, q => true]),
]);

We instantiate a Kripke structure by combining a Kripke frame and the
propositional interpretations over each world.

In [ ]:
K = KripkeStructure(fr, valuation)

Let's generate a random modal formula!

In [ ]:
SoleLogics.BASE_MODAL_CONNECTIVES

In [ ]:
φmodal = randformula(
    Random.MersenneTwister(5678),
    3,      # height
    [p,q],  # alphabet
    SoleLogics.BASE_MODAL_CONNECTIVES
)

In [ ]:
normalize(φmodal)

Let'sheck the formula on each world of the Kripke structure!

In [ ]:
[w => check(φmodal, K, w) for w in worlds]

**Exercise**: check many, randomly-generated *modal* formulas on many,
randomly-generated *modal* interpretations, and store the formulas that satisfy
the highest number of instances!

**Exercise**: define a structure for representing a *modal* interpretation on
scalar variables. You can see if your solution it works by running the cell
below, which instantiates a random modal scalar interpretation.

In [ ]:
module exercise4

export ModalInterpretation

using Main: ConditionOnAttribute

using SoleLogics

using SoleLogics: AbstractFrame, World, AbstractKripkeStructure

# TODO:
# struct ModalInterpretation{FR<:AbstractFrame,T<:Real} <: AbstractKripkeStructure
#     frame::FR
#     vals::???
# end

import SoleLogics: interpret, frame

# Retrieve the interpretation's frame
frame(i::ModalInterpretation) = i.frame

# TODO:
# function interpret(a::Atom{ConditionOnAttribute}, I::ModalInterpretation, w::World)
#     cond = value(a)
#     v = ???
#     return (v < cond.threshold ? TOP : BOT)
# end


end # end module

In [ ]:
using .exercise4

rng = Random.MersenneTwister(1)
n_variables = 4
n_worlds = 5
n_edges = 7
n_formulas = 10

worlds = SoleLogics.World.(1:n_worlds)
g = SimpleDiGraph(n_worlds, n_edges; rng)
fr = SoleLogics.ExplicitCrispUniModalFrame(worlds, g)
variable_values = [rand(n_variables) for w in worlds]

Imodal = ModalInterpretation(fr, variable_values)

for i_formula in 1:n_formulas
    φmodal = randformula(
        Random.MersenneTwister(i_formula),
        2,
        A,
        SoleLogics.BASE_MODAL_CONNECTIVES
    )

    println(
        syntaxstring(φmodal) => [
            "w$(SoleLogics.name(w))" => check(φmodal, Imodal, w) for w in worlds
        ]
    )
    println()
end

### Multi-Modal Logic (i.e., Temporal and Spatial Logics)
`SoleLogics.jl` provides useful aliases for the most common multi-modal logics:
- Linear Temporal Logic with Future and Past (LTL[F,P])
- Compass Logic (CL)
- Halpern and Shoham's Modal Logic of Time Intervals (HS)
- Lutz and Wolter's Modal Logic of Topological Relations with rectangular
  areas aligned with the axes (LRCC8_Rec)

Relation names are given in an uniform way as follows:
  `NAMEOFTHELOGIC`_`NAMEOFTHERELATION`

For example, the relation Future (F) for LTL[F,P] will be named:
  `LTLFP_F`

#### Linear Temporal Logic with Future and Past (LTL[F,P])

In [ ]:
using SoleLogics: LTLFP_F, LTLFP_P  # These aliases are not exported by default

Let's begin by defining a set of 5 ordered points in time.

In [ ]:
fr = FullDimensionalFrame((5,), Point1D{Int})
allworlds(fr) |> collect

Which points are in the future, w.r.t. `3`?

In [ ]:
collect(accessibles(fr, Point(3), LTLFP_F))

Which points are in the past, w.r.t. `3`?

In [ ]:
collect(accessibles(fr, Point(3), LTLFP_P))

Is `5` happening later than `2`?

In [ ]:
Point(5) in accessibles(fr, Point(2), LTLFP_F)

Is `1` happening later than `2`?

In [ ]:
Point(1) in accessibles(fr, Point(2), LTLFP_F)

How many points are later than `2` (over the 5 points we have defined)?

In [ ]:
length(accessibles(fr, Point(2), LTLFP_F))

#### Compass Logic (CL)

In [ ]:
using SoleLogics: CL_N, CL_S, CL_E, CL_W

Let's define a set of 25 points, of coordinates `(x,y)`, in space.

In [ ]:
fr = FullDimensionalFrame((5,5), Point2D{Int})
allworlds(fr) |> collect

Which points are north of `(3,3)`?

In [ ]:
collect(accessibles(fr, Point(3,3), CL_N))

Which points are south of `(3,3)`?

In [ ]:
collect(accessibles(fr, Point(3,3), CL_S))

Which points are east of `(3,3)`?

In [ ]:
collect(accessibles(fr, Point(3,3), CL_E))

Which points are west of `(3,3)`?

In [ ]:
collect(accessibles(fr, Point(3,3), CL_W))

Is `(2,4)` west of `(5,4)`?

In [ ]:
Point(2,4) in accessibles(fr, Point(5,4), CL_W)

Is `(2,4)` north of `(1,3)`?

In [ ]:
Point(2,4) in accessibles(fr, Point(1,3), CL_N)

Is `(2,4)` north of `(2,3)`?

In [ ]:
Point(2,4) in accessibles(fr, Point(2,3), CL_N)

How many points are north of `(4,2)`?

In [ ]:
length(accessibles(fr, Point(4,2), CL_N))

#### Halpern and Shoham's Modal Logic of Time Intervals (HS)

Remember: all relations are mutually disjoint!

(i.e., only one is valid at one time).

In [ ]:
using SoleLogics: HS_A, HS_L, HS_B, HS_E, HS_D, HS_O
using SoleLogics: HS_Ai, HS_Li, HS_Bi, HS_Ei, HS_Di, HS_Oi

Let's generate 10 linearly spaced intervals; to do so, we need 11 points.

Combining them we can get up to 55 intervals!

In [ ]:
fr = FullDimensionalFrame((10,), SoleLogics.Interval{Int})
allworlds(fr) |> collect

Which intervals come after `[4,8]`?

In [ ]:
collect(accessibles(fr, SoleLogics.Interval(4,8), HS_A))

Which come later than `[4,8]`?

In [ ]:
collect(accessibles(fr, SoleLogics.Interval(4,8), HS_L))

Which intervals begin with `[4,8]`, but end before?

In [ ]:
collect(accessibles(fr, SoleLogics.Interval(4,8), HS_B))

Which intervals begin after `[4,8]`, but end with the same point?

In [ ]:
collect(accessibles(fr, SoleLogics.Interval(4,8), HS_E))

Which intervals happen during `[4,8]`?

In [ ]:
collect(accessibles(fr, SoleLogics.Interval(4,8), HS_D))

Which intervals overlap with `[4,8]` but begin later?

In [ ]:
collect(accessibles(fr, SoleLogics.Interval(4,8), HS_O))

Which intervals end exactly when `[4,8]` begins?

In [ ]:
collect(accessibles(fr, SoleLogics.Interval(4,8), HS_Ai))

Which intervals happen before `[4,8]`?

In [ ]:
collect(accessibles(fr, SoleLogics.Interval(4,8), HS_Li))

Which intervals begin with `[4,8]`, but end later?

In [ ]:
collect(accessibles(fr, SoleLogics.Interval(4,8), HS_Bi))

Which intervals end with `[4,8]`, but start before?

In [ ]:
collect(accessibles(fr, SoleLogics.Interval(4,8), HS_Ei))


Which intervals contain `[4,8]`?

In [ ]:
collect(accessibles(fr, SoleLogics.Interval(4,8), HS_Di)) 

Which intervals overlap with `[4,8]` but begin before?

In [ ]:
collect(accessibles(fr, SoleLogics.Interval(4,8), HS_Oi))

#### Lutz and Wolter's Modal Logic of Topological Relations with rectangular areas aligned with the axes (LRCC8_Rec)

In [ ]:
using SoleLogics: LRCC8_Rec_DC, LRCC8_Rec_EC, LRCC8_Rec_PO
using SoleLogics: LRCC8_Rec_TPP, LRCC8_Rec_TPPi, LRCC8_Rec_NTPP, LRCC8_Rec_NTPPi

Let's create 5 linearly spaced intervals on both axes (i.e., 6 points on each).

We can combine them in 225 different ways to define rectangles!

Note that rectangles are specified using a tuple of intervals, one on each axis,
e.g., `([2,4], [3,5])` identifies a rectangle using `Point`s `2` and `4` from
the `x` axis and `3` and `5` from the `y` axis.

In [ ]:
fr = FullDimensionalFrame((5,5), SoleLogics.Interval2D{Int})
allworlds(fr) |> collect

How many rectangles can I build so that they are disconnected from
`([3,4], [3,4])`?

In [ ]:
length(
    collect(accessibles(fr, SoleLogics.Interval2D((3,4),(3,4)), LRCC8_Rec_DC))
)

How many rectangles can I define so that they are inside `([2,5], [2,5])` and
touching it on the perimeter? (i.e., tangential proper part)

In [ ]:
length(
    collect(accessibles(fr, SoleLogics.Interval2D((2,5),(2,5)), LRCC8_Rec_TPP))
)

How many rectangles can I define so that they contain `([3,4], [3,4])` not
touching it on the perimeter? (i.e., non-tangential proper part inverse)

In [ ]:
length(
    collect(accessibles(fr, SoleLogics.Interval2D((3,4),(3,4)), LRCC8_Rec_NTPPi))
)

Let's create another frame consisting of 3 linearly spaced intervals on both
axes (i.e., 4 points on each).

We can combine them in 36 different ways to define rectangles!

In [ ]:
fr = FullDimensionalFrame((3,3), SoleLogics.Interval2D{Int})
allworlds(fr) |> collect

How many rectangles are externally connected with `([2,3],[2,3])`?

In [ ]:
length(
    collect(accessibles(fr, SoleLogics.Interval2D((2,3),(2,3)), LRCC8_Rec_EC))
)

Let's create another frame consisting of 4 linearly spaced intervals on both
axes (i.e., 5 points on each).

We can combine them in 100 different ways to define rectangles!

In [ ]:
fr = FullDimensionalFrame((4,4), SoleLogics.Interval2D{Int})
allworlds(fr) |> collect

How many rectangles are partially overlapping with `([2,4],[2,4])`?

In [ ]:
length(
    collect(accessibles(fr, SoleLogics.Interval2D((2,4),(2,4)), LRCC8_Rec_PO))
)

How many rectangles contain `([2,4],[2,4])` and touch it on the perimeter?

In [ ]:
length(
    collect(accessibles(fr, SoleLogics.Interval2D((2,4),(2,4)), LRCC8_Rec_TPPi))
)

Let's create a final frame consisting of 6 linearly spaced intervals on both
axes (i.e., 7 points on each).

We can combine them in 441 different ways to define rectangles!

In [ ]:
fr = FullDimensionalFrame((6,6), SoleLogics.Interval2D{Int})
allworlds(fr) |> collect

How many rectangles are contained in `([2,6], [2,6])` without touching it on the
perimeter?

In [ ]:
length(
    collect(accessibles(fr, SoleLogics.Interval2D((2,6),(2,6)), LRCC8_Rec_NTPP))
)

### Fuzzy Logic

In standard fuzzy logics, instead of constraining ourselves to only `true` and
`false` values, we let them be anythig between the continuous interval [0, 1].

On the downside, we cannot use the classical evaluation for the propositional
operators - what does it mean to be `0.3 and 0.5`?

Fuzzy logics are defined over a `t-norm` operation, which will be our
conjunction ($\wedge$); we have 3 of them(*):
- Goedel Logic, where the $tnorm(x, y)$ is defined as the $min\{x, y\}$
- Lukasiewicz Logic, where the $tnorm(x, y)$ is defined as the
$max\{0, x+y-1\}$
- Product Logic, where the $tnorm(x, y)$ is defined as the arithmetic product
$x \cdot y$

For each logic, the implication $x \to y$ will be defined as the
$max\{z | tnorm(x, z) \leq y\}$.

This can feel overwhelming at first, as it is very general: the important part
is that we can derive the implication for each fuzzy logic from their `t-norm`.

Finally, we will consider the disjunction between two values $x$ and $y$ simply
as the $max\{x, y\}$ for all logics.

(*) All other t-norms (hence, fuzzy logics) can be derived though a linear
combination of these 3.

In [ ]:
using SoleLogics.ManyValuedLogics   # fuzzy logics are defined in this submodule

In [ ]:
GodelLogic

In [ ]:
bot(GodelLogic)

In [ ]:
top(GodelLogic)

In [ ]:
uknown = ContinuousTruth(0.5)

The $tnorm(x, y)$ for Goedel Logic is defined as the $min\{x, y\}$.

In [ ]:
GodelLogic.tnorm(
    uknown,
    uknown
)

In [ ]:
LukasiewiczLogic

The $tnorm\{x, y\}$ for Lukasiewicz Logic is defined as the $max\{0, x+y-1\}.

In [ ]:
LukasiewiczLogic.tnorm(
    uknown,
    uknown
)

The $tnorm\{x, y\}$ for Product Logic is defined as the arithmetic product
$x \cdot y$.

In [ ]:
ProductLogic.tnorm(
    uknown,
    uknown
)

### Many-Valued Logic

While in fuzzy logics we consider a total order between all the values,
many-valued logics go even a step further: we also take into consideration
values which can be non-comparable, i.e., partial orders.

To do so, we leverage algebraic structures comprising lattices, such as Heyting
Algebras.

In [ ]:
using SoleLogics.ManyValuedLogics: G4, Ł4, H4
using SoleLogics.ManyValuedLogics: α, β

In [ ]:
getdomain(G4)

In [ ]:
getdomain(Ł4)

In [ ]:
getdomain(H4)

In [ ]:
precedes(G4, α, β)

In [ ]:
precedes(Ł4, α, β)

In [ ]:
precedes(H4, α, β)

In [ ]:
precedes(H4, ⊥, α)

In [ ]:
precedes(H4, ⊥, β)

In [ ]:
precedes(H4, α, ⊤)

In [ ]:
precedes(H4, β, ⊤)

While fuzzy logics differ on the `t-norm`, many-valued logics in general are
defined, among other things (like the set of values), over a more general
structure, called a `monoid`, that we will use to interpret conjunction
($\wedge$).

This coincide with the `t-norm` for fuzzy logics.

In [ ]:
G4.monoid(α, β) # min(α, β)

In [ ]:
Ł4.monoid(α, β) # max(α+β-1, 0)

Heyting algebras can be thought of as a generalization of Goedel algebras (resp.
Goedel Logic) to partial orders: instead of taking the $min\{x, y\}$, which is
not always possible, we take the $inf\{x, y\}$, i.e., the greatest lower bound.

For instance, since in our case $\alpha$ and $\beta$ are non-comparable, but
both greater than $\bot$ (and bigger than $\top$), their $inf$ will be $\bot$.

An $inf$ and a $sup$ is always guaranteed to exist (and to be unique) by
definition of a lattice.

In [ ]:
H4.monoid(α, β) # inf(α, β)

Let's have a quick look at a more complex example.

The following is also an Heyting algebras, but with 9 values.

In [ ]:
using SoleLogics.ManyValuedLogics: H9
using SoleLogics.ManyValuedLogics: ζ, η

getdomain(H9)

Let's take, for example, values $\zeta$ and $\eta$: they are non-comparable,
but they are both bigger than $\bot, \alpha, \beta, \delta$.

In [ ]:
using SoleLogics.ManyValuedLogics: lesservalues

lesservalues(H9, ζ)

In [ ]:
lesservalues(H9, η)

In [ ]:
intersect(lesservalues(H9, ζ), lesservalues(H9, η))

Hence, the greatest of these values ($\delta$) will be our result.

In [ ]:
H9.monoid(ζ, η)

**Exercise**: try to do the same with other values from the H9 algebra, such as:
- $\zeta$ and $\epsilon$
- $\zeta$ and $\beta$

Disjunction ($\vee$) and implication ($\to$) are generalized in a similar way:
- to evaluate a disjunction $x \vee y$, we will use the $sup\{x, y\}$ (lowest
greater bound)
- to evaluate an implication $x \to y$, we will use the
$sup\{z | monoid(x, z) \preceq y\}$

Let's focus on the latter with an example.

In [ ]:
x = ζ
y = η

candidates = Vector{FiniteTruth}()

for z in getdomain(H9)
    r = H9.monoid(x, z)
    print("$x⋅$z=$r")
    print("\t$r⪯$y")
    if precedeq(H9, r, y)
        print("\t✅")
        push!(candidates, z)    # WATCH OUT! I'm pushing z, NOT r!
        println("\tPushing $z to the list of candidates...")
    else
        println("\t❌")
    end
end

println("\ncandidates: $candidates")

In [ ]:
candidate = first(candidates)
for new_candidate in candidates[2:length(candidates)]
    if precedes(H9, candidate, new_candidate)   # candidate ≺ new_candidate
        candidate = new_candidate
    end
end
result = candidate


In [ ]:
H9.implication(ζ, η)

**Exercise**: try to do the same with other values from the H9 algebra, such as:
- $\zeta$ and $\epsilon$
- $\zeta$ and $\beta$

And of course, we can also use the `check`...

In [ ]:
model = TruthDict([p=>α, q=>β])

In [ ]:
check(parseformula("p∧q"), model, G4)   # return true only if ⊤

and the `interpret`!

In [ ]:
interpret(parseformula("p∧q"), model, G4)   # min

Some other examples...

In [ ]:
check(parseformula("p∨q"), model, H4)

In [ ]:
interpret(parseformula("p∨q"), model, H4)   # sup

In [ ]:
model = TruthDict([p=>ζ, q=>η])

In [ ]:
interpret(parseformula("p∧q"), model, H9)

In [ ]:
interpret(parseformula("p→q"), model, H9)

**Esercise**: try the same exercises from before, but this time use the
`interpret` function instead.

## SoleReasoners.jl

One classical problem we would like to solve is satisfiability (SAT).

I.e., given a formula in a specific logic, is there a model that satisfies
that model?

The literature provides us with many tools to do that, called SAT solvers.

`Sole.jl` provides its own package, called `SoleReasoners.jl`, which works out
of the box with `SoleLogic.jl`'s syntax.

In [ ]:
Pkg.add(url="https://github.com/aclai-lab/SoleReasoners.jl#embedding")

In [ ]:
using SoleReasoners

In [ ]:
p, q = Atom.(["p", "q"])

In [ ]:
φ = parseformula("p∧q");
sat(φ)

In [ ]:
φ = parseformula("p∧¬p")
sat(φ)

Similarly, we can ask if a formula is valid (i.e., satisfied by any possible
model).

Once again, the literature provides us with many tools, namely automated theorem
provers.

`SoleReasoners.jl` uses an analytic tableau technique to find if a formula is
satistiable or not: hence, if we want to ask if a formula is valid or not, we
just have to negate the input formula and searching for a countermodel (hence,
we have to invert the input).

In [ ]:
using SoleReasoners: prove as val   # function isn't exported by default

In [ ]:
φ = parseformula("p∧q");
val(φ)

In [ ]:
φ = parseformula("p∨⊤");
val(φ)

`SoleReasoners.jl` offers also support for Modal Logic K; however, it doesn't
define an algorithm explicitly, but rather levarage an already existing sat
solver (*spartacus*).

In the sake of time, we won't see it here (we would have to install this other
third party tool).

On the other hand, it also supports the four multi-modal logic saw above, as
well as (finite) fuzzy and many-valued logics!

Well... actually it supports something more general than that... which is a
combination of multi-modal and many-valued logics!

Don't stress too much about it: we can still use it for either (crisp)
multi-modal or propositional many-valued logic!

In [ ]:
# Let's introduce some aliases in the hope to create less confusion
LTLFPTableau = MVLTLFPTableau
CLTableau = MVCLTableau
HSTableau = MVHSTableau
LRCC8Tableau = MVLRCC8Tableau

# For many-valued logic (including fuzzy logic), we could use any of the 4 above
MVTableau = MVLTLFPTableau
;

Let's start by defining some atoms.

In [ ]:
p, q, r, s = @atoms p q r s

Let's see some examples with HS...

First, we need to define the diamond and box operators for the relations we'd
like to use.

In [ ]:
diamondA = diamond(HS_A)    # diamond for after (at leat one interval after...)
boxA = box(HS_A)            # box for after (all intervals after...)

Let's try the following formula (for now, we will prefer functional notation).

In [ ]:
φ = ∧(diamondA(p), boxA(→(p, ⊥)))   # this is, of course, a contraddiction!

Even if we are just interested in the crisp version of the logic, we have to
use a function called `alphasat`, which name will be clear in a minute.

Moreover, we have to specify which kind of tableau we are using, given in the
form `LOGICTableau`, and the many-valued algebra we are using; since we are in
the crisp case, we will use the `booleanalgebra`.

In [ ]:
using SoleLogics.ManyValuedLogics: booleanalgebra

alphasat(HSTableau, ⊤, φ, booleanalgebra)   # Don't mind the ⊤: we need it

We can check that the formula is in fact a contraddiction in the following way:

In [ ]:
alphaval(HSTableau, ⊤, →(φ, ⊥), booleanalgebra)

What about many-valued logics?

`alphasat` is in fasct a generalization of the `sat` problem: we are no longer
asking if there exists a model so that is satisfies a formula, but rather if
there is one so that the valuation of the formula is, at least, some specified
value $\alpha$ from the algebra.

That's why we needed to specify $\top$ in the previous example!

Here's an example using Goedel Logic with 3 values...

In [ ]:
using SoleLogics.ManyValuedLogics: G3

φ = →(∨(p, ⊤), α)

Of course, the $p \vee \top$ subformula is always true in G3.

In [ ]:
alphaval(MVTableau, ⊤, ∨(p, ⊤), G3)

So, the previous formula could've been written $\top \to \alpha$; it's just
syntax sugar.

On the other hand, $\top \to \alpha$ in G3 equals $\alpha$.

In [ ]:
G3.implication(⊤, α)

That means that `sat` for this formula would return false!

In [ ]:
alphasat(MVTableau, ⊤, φ, G3)

But what if we asked that the valuation was at least $\alpha$?

In [ ]:
alphasat(MVTableau, α, φ, G3)

Same goes for `alphaval` (given any model, the valuation of the formula is at
least $\alpha$).

In [ ]:
alphaval(MVTableau, α, φ, G3)   # This was an wasy case: φ doedn't depend on p

**Exercise**: play with the other logics (LTLFP, CL, LRCC8) and the other
algebras (G4, Ł3, Ł4, H4, H9).